In [1]:
import os
from delta import *

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages io.delta:delta-spark_2.12:3.2.0 "
    "--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension "
    "--conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell"
)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DeltaLakeExample").master("spark://localhost:7077")
spark = configure_spark_with_delta_pip(spark).getOrCreate()

25/05/13 19:55:08 WARN Utils: Your hostname, Ubuntu-Home resolves to a loopback address: 127.0.1.1; using 192.168.1.146 instead (on interface eno1)
25/05/13 19:55:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/mparcerisa/Documents/Work/MDS/S2/BDM/Project/PROJECT/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mparcerisa/.ivy2/cache
The jars for the packages stored in: /home/mparcerisa/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8ac59112-b587-43f8-9f4a-9cd9db47dc6c;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 135ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3 

In [3]:
from pyspark.sql import Row

data = [
    Row(id=1, name="Alice", age=29),
    Row(id=2, name="Bob", age=31),
    Row(id=3, name="Cathy", age=25),
    Row(id=4, name="David", age=35),
    Row(id=5, name="Eve", age=28),
]
data = data * 100

df = spark.createDataFrame(data)
display(df)

DataFrame[id: bigint, name: string, age: bigint]

In [4]:
print(spark.version)
print(spark.conf.get("spark.sql.extensions", None))
print(spark.conf.get("spark.sql.catalog.spark_catalog", None))

3.5.1
io.delta.sql.DeltaSparkSessionExtension
org.apache.spark.sql.delta.catalog.DeltaCatalog


In [5]:
df.write.format("delta").mode("overwrite").save("/tmp/landing_test")

25/05/13 19:55:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
df = spark.read.format("delta").load("/tmp/landing_test")
df.show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  2|  Bob| 31|
|  3|Cathy| 25|
|  4|David| 35|
|  5|  Eve| 28|
|  1|Alice| 29|
|  2|  Bob| 31|
|  3|Cathy| 25|
|  4|David| 35|
|  5|  Eve| 28|
|  1|Alice| 29|
|  2|  Bob| 31|
|  3|Cathy| 25|
|  4|David| 35|
|  5|  Eve| 28|
|  1|Alice| 29|
|  2|  Bob| 31|
|  3|Cathy| 25|
|  4|David| 35|
|  5|  Eve| 28|
|  1|Alice| 29|
+---+-----+---+
only showing top 20 rows

